In [1]:
from difflogic import LogicLayer, GroupSum
import torch
from chop import MaseGraph

/rds/general/user/jj21/home/miniforge3/envs/adls-project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# import sys

# "Add dot (pydot) to path if needed:"

# new_path = os.path.expanduser("~/miniforge3/envs/adls-project/bin/")
# if new_path not in sys.path:
#     sys.path.append(new_path)

# # Add to environment PATH as well
# os.environ["PATH"] = new_path + os.pathsep + os.environ["PATH"]

In [3]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    LogicLayer(400, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    GroupSum(k=10, tau=20)
)

In [24]:
CUSTOM_OPS = {
    "modules": {
        LogicLayer: {
            "args": {"input":"data_in"},
        },
        GroupSum: {
            "args": {"input":"data_in"},
        },
    },
    "functions": {},
}

mg = MaseGraph(model, custom_ops=CUSTOM_OPS)

# mg.draw("DLG.svg")

In [34]:
def eval(model, loader, mode):
    orig_mode = model.training
    with torch.no_grad():
        model.train(mode=mode)
        res = np.mean(
            [
                (model(x.to('cpu').round()).argmax(-1) == y.to('cpu')).to(torch.float32).mean().item()
                for x, y in loader
            ]
        )
        model.train(mode=orig_mode)
    return res.item()

In [35]:
import mnist_dataset
import numpy as np

train_set = mnist_dataset.MNIST('./data-mnist', train=True, download=True, remove_border=True)
test_set = mnist_dataset.MNIST('./data-mnist', train=False, remove_border=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, pin_memory=True, drop_last=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, pin_memory=True, drop_last=True)

state_dict = torch.load("best_model_full.pth", map_location=torch.device('cpu'))  # Load the dictionary
mg.model.load_state_dict(state_dict, strict=False)  # Apply the weights

# mg.model.eval()

# print("Accuracy: ", eval(mg.model, test_loader, mode=False))

_IncompatibleKeys(missing_keys=['7.k', '7.tau'], unexpected_keys=[])

In [36]:
import types

def test_pass(
    graph,
    pass_args={"model":"None", "state_dict":{}},
):
#     model = pass_args["model"]
#     given_params = model.state_dict()
#     if (pass_args["state_dict"]):
#         given_params = pass_args["state_dict"]
    
    
    # Step 1: Copy input placeholder nodes
    for node in graph.nodes:
        print(node.op)
        print(node.meta["mase"].parameters)
        print()
#         if "LogicLayer" in node.op:  # Detect input nodes
            
    

    return mg, None
                    
                    
                    
#     nodes = set({})
#     for node in mg.fx_graph.nodes:
#         if (node.op == "call_module"):
#             pass
#         elif (node.op == "call_method" and long not in node.target ):
#             if len(node.args) > 1:
#                 raise "What?"
#             node.replace_all_uses_with(node.args[0])
#         elif (isinstance(node.target, types.BuiltinFunctionType):
#              node.replace_all_uses_with(node.args[0]) 
            
            
#     return graph, None

In [39]:
import chop.passes as passes

dummy_in_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=True)
image, label = next(iter(dummy_in_loader))
# image = image.cpu().squeeze(0) 

mg_test, _ = passes.init_metadata_analysis_pass(mg)
mg_test, _ = passes.add_common_metadata_analysis_pass(
    mg_test,
    pass_args={
        "dummy_in": {"input_1": image}, #input name Hardcoded here, change
#         "add_value": False,
    },
)

# There is error below, need to fix
# mg_test, _ = passes.add_hardware_metadata_analysis_pass(mg_test)


mg_test.draw("DLG_test.svg")


# below pass just prints out the metadata for us to debug
test_pass(mg)

placeholder
{'common': {'mase_type': 'placeholder', 'mase_op': 'placeholder', 'args': {}, 'results': OrderedDict([('data_out_0', {'type': 'float', 'precision': [32], 'shape': [1, 1, 20, 20], 'torch_dtype': torch.float32, 'value': tensor([[[[0.0000, 0.0000, 0.3294, 0.7255, 0.6235, 0.5922, 0.2353, 0.1412,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.8706, 0.9961, 0.9961, 0.9961, 0.9961, 0.9451,
           0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765,
           0.6667, 0.2039, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.2627, 0.4471, 0.2824, 0.4471, 0.6392, 0.8902,
           0.9961, 0.8824, 0.9961, 0.9961, 0.9961, 0.9804, 0.8980, 0.9961,
           0.9961, 0.5490, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0667,
           0.2588, 0.0549, 0.2627, 0.2627, 0.2627, 0.2314, 0.0824, 0.9255,
           0.9961, 0.4157, 0.0000, 0.0

(<chop.ir.graph.mase_graph.MaseGraph at 0x149a5192d8d0>, None)